In [20]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
from keras.utils import to_categorical
import cv2, numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import applications
from PIL import Image
Image.LOAD_TRUNCATED_IMAGES = True
import xml.etree.cElementTree as ET
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from shutil import copyfile
from pathlib import Path
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10185302795862381013
, name: "/device:GPU:0"
device_type: "GPU"
locality {
  bus_id: 1
}
incarnation: 3570933478606427533
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


# Test VGG16

In [27]:
nbClasses = 5
img_rows, img_cols, img_channel = 224, 224, 3

base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, img_channel))

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(nbClasses, activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
for layer in base_model.layers:
    layer.trainable = False
    
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,128,256]
	 [[Node: block3_conv1_8/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=2028524, _device="/job:localhost/replica:0/task:0/device:GPU:0"](block3_conv1_8/random_uniform/shape)]]

Caused by op 'block3_conv1_8/random_uniform/RandomUniform', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/py

In [26]:
data_path = '/data/'

batch_size = 8
epochs = 8

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    data_path + 'Training/',  
    target_size=(224,224),  
    batch_size=batch_size)  


validation_generator = test_datagen.flow_from_directory(
        data_path + 'Validation/',
        target_size=(224,224),
        batch_size=batch_size)

model.fit_generator(
    train_generator,
    steps_per_epoch= 299 // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[ModelCheckpoint(data_path + 'VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)]
)
#model.save_weights(data_path + 'weights/try_VGG16.h5')

model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.0001, momentum=0.9),
              metrics=['accuracy'])
              
model.fit_generator(
    train_generator,
    steps_per_epoch= 299 // batch_size,
    epochs=2,
    validation_data=validation_generator,
    callbacks=[ModelCheckpoint(data_path + 'VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)]
)

Found 801547 images belonging to 10000 classes.
Found 343492 images belonging to 10000 classes.


NameError: name 'model' is not defined

In [17]:
print(predictions)

[[ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]]
